To proceed, I’ll now give you a full working implementation step-by-step for a basic Seq2Seq Encoder-Decoder model with LSTMs in PyTorch using a toy English-to-Hindi dataset.

✅ Step-by-Step LSTM Encoder-Decoder Seq2Seq in PyTorch (Working Code + Use Case)
📌 1. Install Required Packages


https://www.kaggle.com/datasets/preetviradiya/english-hindi-dataset

In [ ]:
# python version
# library version


In [7]:
# pip install torch torchtext
# Uninstall existing torch and torchtext
!pip uninstall torch torchtext -y

# Install torch and torchtext with compatible versions
# You might need to adjust the version and the cuXXXX part based on your system and desired CUDA version
# For example, for CUDA 11.8, use cu118
# For CPU only, use torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install torch==2.2.0+cu118 torchtext

ERROR: Could not find a version that satisfies the requirement torch==2.2.0+cu118 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0)
ERROR: No matching distribution found for torch==2.2.0+cu118


In [9]:
# After reinstalling, rerun the rest of your code starting from the imports.
import torch
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

data = [
    ("hello", "नमस्ते"),
    ("how are you", "आप कैसे हैं"),
    ("i am fine", "मैं ठीक हूँ"),
    ("thank you", "धन्यवाद"),
    ("what is your name", "आपका नाम क्या है"),
    ("my name is john", "मेरा नाम जॉन है"),
]

# Rest of your code...

ModuleNotFoundError: No module named 'torch'

In [11]:
# !pip list

In [12]:
# After reinstalling, rerun the rest of your code starting from the imports.
import torch
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

data = [
    ("hello", "नमस्ते"),
    ("how are you", "आप कैसे हैं"),
    ("i am fine", "मैं ठीक हूँ"),
    ("thank you", "धन्यवाद"),
    ("what is your name", "आपका नाम क्या है"),
    ("my name is john", "मेरा नाम जॉन है"),
]

# Rest of your code...

ModuleNotFoundError: No module named 'torch'

In [13]:
# After reinstalling, rerun the rest of your code starting from the imports.
import torch
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

data = [
    ("hello", "नमस्ते"),
    ("how are you", "आप कैसे हैं"),
    ("i am fine", "मैं ठीक हूँ"),
    ("thank you", "धन्यवाद"),
    ("what is your name", "आपका नाम क्या है"),
    ("my name is john", "मेरा नाम जॉन है"),
]

# Rest of your code...

ModuleNotFoundError: No module named 'torch'

In [5]:
# After reinstalling, rerun the rest of your code starting from the imports.
import torch
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

data = [
    ("hello", "नमस्ते"),
    ("how are you", "आप कैसे हैं"),
    ("i am fine", "मैं ठीक हूँ"),
    ("thank you", "धन्यवाद"),
    ("what is your name", "आपका नाम क्या है"),
    ("my name is john", "मेरा नाम जॉन है"),
]

# Rest of your code...

ModuleNotFoundError: No module named 'torch'

In [ ]:
# 📌 2. Prepare Dataset

import torch
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

data = [
    ("hello", "नमस्ते"),
    ("how are you", "आप कैसे हैं"),
    ("i am fine", "मैं ठीक हूँ"),
    ("thank you", "धन्यवाद"),
    ("what is your name", "आपका नाम क्या है"),
    ("my name is john", "मेरा नाम जॉन है"),
]


In [ ]:
# Tokenizers
en_tokenizer = get_tokenizer("basic_english")
hi_tokenizer = get_tokenizer("basic_english")

def yield_tokens(data, tokenizer, idx):
    for pair in data:
        yield tokenizer(pair[idx])

SRC_VOCAB = build_vocab_from_iterator(yield_tokens(data, en_tokenizer, 0), specials=["<pad>", "<sos>", "<eos>"])
TGT_VOCAB = build_vocab_from_iterator(yield_tokens(data, hi_tokenizer, 1), specials=["<pad>", "<sos>", "<eos>"])

SRC_VOCAB.set_default_index(SRC_VOCAB["<pad>"])
TGT_VOCAB.set_default_index(TGT_VOCAB["<pad>"])

def tensorize(pair):
    src = [SRC_VOCAB["<sos>"]] + [SRC_VOCAB[tok] for tok in en_tokenizer(pair[0])] + [SRC_VOCAB["<eos>"]]
    tgt = [TGT_VOCAB["<sos>"]] + [TGT_VOCAB[tok] for tok in hi_tokenizer(pair[1])] + [TGT_VOCAB["<eos>"]]
    return torch.tensor(src), torch.tensor(tgt)

In [ ]:
# 📌 3. Create Encoder and Decoder

import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, (hidden, cell) = self.lstm(embedded)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.embedding(input)
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

In [ ]:
# 📌 4. Seq2Seq Training Loop

import torch.optim as optim

INPUT_DIM = len(SRC_VOCAB)
OUTPUT_DIM = len(TGT_VOCAB)
HID_DIM = 256
EMB_DIM = 128

enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
enc, dec = enc.to(device), dec.to(device)

optimizer = optim.Adam(list(enc.parameters()) + list(dec.parameters()))
criterion = nn.CrossEntropyLoss(ignore_index=TGT_VOCAB["<pad>"])


In [ ]:
# Training loop
for epoch in range(100):
    epoch_loss = 0
    for src, tgt in map(tensorize, data):
        src, tgt = src.to(device), tgt.to(device)
        optimizer.zero_grad()
        hidden, cell = enc(src.unsqueeze(1))  # [src_len, batch_size]
        input = tgt[0]
        loss = 0
        for t in range(1, len(tgt)):
            output, hidden, cell = dec(input, hidden, cell)
            loss += criterion(output, tgt[t].unsqueeze(0))
            input = tgt[t]
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if epoch % 10 == 0:
        print(f"Epoch {epoch} Loss: {epoch_loss:.2f}")

In [ ]:
# 📌 5. Translate New Sentences (Inference)

def translate_sentence(sentence):
    enc.eval()
    dec.eval()
    tokens = [SRC_VOCAB["<sos>"]] + [SRC_VOCAB[token] for token in en_tokenizer(sentence)] + [SRC_VOCAB["<eos>"]]
    src_tensor = torch.tensor(tokens).to(device).unsqueeze(1)
    hidden, cell = enc(src_tensor)
    input = torch.tensor([TGT_VOCAB["<sos>"]]).to(device)

    result = []
    for _ in range(20):
        output, hidden, cell = dec(input, hidden, cell)
        top1 = output.argmax(1)
        if top1.item() == TGT_VOCAB["<eos>"]:
            break
        result.append(top1.item())
        input = top1

    translated = [TGT_VOCAB.get_itos()[idx] for idx in result]
    return " ".join(translated)


In [ ]:
# Example
print(translate_sentence("hello"))
print(translate_sentence("what is your name"))
# ✅ Final Output (Sample)

# > hello
# नमस्ते

# > what is your name
# आपका नाम क्या है
